# Mission to Mars

## Step 1 - Scraping

In [39]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import pandas as pd

In [40]:
# Initialize PyMongo to work with MongoDBs
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [41]:
# Define database and collection
db = client.mtm_db
collection = db.news

In [42]:
executable_path = {"executable_path":ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - There is no [win32] chromedriver for browser 95.0.4638 in cache
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\ivana\.wdm\drivers\chromedriver\win32\95.0.4638.54]


### Mars News Site

In [50]:
# URL of page to be scraped
mn_url = "https://redplanetscience.com/"

browser.visit(mn_url)
html=browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, "html.parser")

In [51]:
soup

<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</spa

In [53]:
# Retrieve the latest news title
news_title=soup.find_all("div", class_="content_title")[0].text
# Retrieve the latest news paragraph
news_para=soup.find_all("div", class_="article_teaser_body")[0].text
print(news_title)
print(news_para)

Mars 2020 Unwrapped and Ready for More Testing
In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.


### JPL Mars Space Images - Featured Image

In [54]:
# URL of page to be scraped
jpl_url="https://spaceimages-mars.com"

browser.visit(jpl_url)
html=browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, "html.parser")

In [55]:
soup

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [67]:
# Find the featured image
image_url=soup.find("img", class_="headerimage fade-in").attrs["src"]
image_url

'image/featured/mars3.jpg'

In [66]:
# Combine the jpl_url with the image_url
featured_image_url=jpl_url+"/"+image_url
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [71]:
# URL of page to be scraped
mf_url="https://galaxyfacts-mars.com/"

# Read the tables on the page using Pandas. Viewed each table (element) and found table of interest
# was element 1, therefore tables[1].
tables=pd.read_html(mf_url)
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [72]:
# Store mars facts in a table, rename the columns.
mars_facts=tables[1]
mars_facts_final=mars_facts.rename(columns={0:"Parameter",1:"Value"})
mars_facts_final.set_index("Parameter",inplace=True)
mars_facts_final

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [78]:
# Convert mars_facts_final table to html table string.
mars_table_html=mars_facts_final.to_html()
print(mars_table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Parameter</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [79]:
# URL of page to be scraped
mh_url = "https://marshemispheres.com/"

browser.visit(mh_url)
html=browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, "html.parser")

In [80]:
soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [81]:
# Extract the item elements. Class = "collapsible results" contains the full list and class = "items"
# contains the individual items in which the image and header are stored.
mars_list=soup.find("div",class_="collapsible results")
mars_item=mars_list.find_all("div",class_="item")

In [87]:
# Create an empty list to store the images and url's.
hems_image_urls=[]

# Loop through each item in mars_items
for item in mars_item:
    # Error handling
    try:
        # Extract title
        hem=item.find("div",class_="description")
        title=hem.h3.text
        # Extract image url
        hem_url=hem.a["href"]
        browser.visit(mh_url+hem_url)
        html=browser.html
        soup=BeautifulSoup(html,"html.parser")
        image_src=soup.find("li").a["href"]
        image_url=mh_url+image_src
        if (title and image_url):
            # Print results
            print("------------------------------------------")
            print(title)
            print(image_url)
        # Create dictionary for title and url
        hems_dict={
            "title":title,
            "image_url":image_url
        }
        hems_image_urls.append(hems_dict)
    except Exception as e:
        print(e)

------------------------------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
------------------------------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
------------------------------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
------------------------------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


### Combined Dictionary

In [88]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    "news_paragraph":news_para,
    "featured_image":featured_image_url,
    "fact_table":mars_table_html,
    "hemisphere_images":hems_image_urls
}

mars_dict

{'news_title': 'Mars 2020 Unwrapped and Ready for More Testing',
 'news_paragraph': "In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.",
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars3.jpg',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Parameter</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>